In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

In [8]:
# Cài đặt cấu hình User-Agent 
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_page_url(base_url, page_number):
    """
    Tạo URL cho từng trang theo cấu trúc '-pageX' của Vietnamnet.
    """
    if page_number <= 1:
        return base_url
    return f"{base_url}-page{page_number}"

def scrape_article_body(article_url):
    """
    Truy cập vào URL chi tiết của bài báo và trích xuất nội dung chính (body text).
    """
    try:
        response = requests.get(article_url, headers=HEADERS, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Thử tìm kiếm container chứa nội dung bài viết chính.
        content_container = soup.find('div', class_='article__body') or \
                            soup.find('div', class_='content-detail') or \
                            soup.find('div', id='inner-article')
                            
        if not content_container:
            return "KHÔNG TÌM THẤY CONTAINER NỘI DUNG"
        else:
            paragraphs = content_container.find_all('p')
            if paragraphs:
                paragraphs.pop(0)

            full_text = '\n'.join([p.get_text(" ",strip=True) for p in paragraphs if p.get_text(strip=True) and not p.find_parent('figcaption')])
        # Loại bỏ các chuỗi rỗng và khoảng trắng thừa
        return full_text.strip()
        
    except requests.exceptions.RequestException as e:
        return "LỖI KẾT NỐI"

def scrape_page(url, current_id):
    """
    Tìm kiếm và lấy nội dung bài viết (Tiêu đề, URL) từ một trang danh sách, 
    sau đó gọi scrape_article_body để lấy nội dung chi tiết.
    """
    print(f"--- Đang tải dữ liệu từ trang danh sách: {url}")

    articles_data = []

    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        response.raise_for_status() 

        soup = BeautifulSoup(response.content, 'html.parser')
        

        # 1. Tìm tất cả các container có thể chứa danh sách bài viết
        list_containers = [soup]

        article_list_data = []

        unique_urls = set()

        for container in list_containers:
            # 2. Tìm tất cả các tiêu đề chứa link bài viết
            title_tags = container.find_all('h3')
            
            for tag in title_tags:
                link = tag.find('a')
                
                if link and link.get('href') and link.get_text(strip=True):
                    title = link['title']
                    href = link['href']

                    full_url = "https://vietnamnet.vn" + href if href.startswith('/') else href
                        
                    if full_url not in unique_urls:
                        article_list_data.append({
                            'Title': title,
                            'URL': full_url,
                        })
                        unique_urls.add(full_url)
        
        if not article_list_data:
            print("Cảnh báo: Không tìm thấy bất kỳ liên kết bài viết nào trên trang này.")


        # BƯỚC: Truy cập từng bài viết để lấy nội dung đầy đủ và gán ID
        for idx, article in enumerate(article_list_data):
            print(f"  > Lấy nội dung bài viết {idx+1}/{len(article_list_data)}: {article['Title'][:50]}...")
            
            body_text = scrape_article_body(article['URL'])
            
            current_id += 1 
            
            articles_data.append({
                'ID': current_id,
                'URL': article['URL'],
                'Title': article['Title'],
                'Content': body_text,
            })

            time.sleep(2)
            

        return articles_data, current_id
                 
    except requests.exceptions.RequestException as e:
        print(f"Lỗi kết nối khi tải {url}: {e}")

    return articles_data, current_id

def save_to_csv(data, filename):
    """
    Lưu dữ liệu đã crawl vào tệp CSV, chỉ bao gồm ID, URL, Tiêu đề và Nội dung.
    """
    if not data:
        print("Không có dữ liệu để lưu.")
        return

    fieldnames = ['ID', 'URL', 'Title', 'Content']
    
    column_mapping = {
        'ID': 'ID',
        'URL': 'URL',
        'Title': 'Tiêu đề',
        'Content': 'Nội dung Bài báo',
    }
    
    try:
        # Sử dụng utf-8-sig để thêm BOM, giúp Excel mở đúng mã hóa tiếng Việt
        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writerow({k: column_mapping[k] for k in fieldnames})
            
            writer.writerows(data)
            
        print(f" Đã hoàn tất! Đã lưu tổng cộng {len(data)}")
    except Exception as e:
        print(f"Lỗi khi lưu tệp CSV: {e}")


def main():
    BASE_URL = "https://vietnamnet.vn/van-hoa-giai-tri/di-san"
    NUM_PAGES = 25
    all_articles = []
    current_id = 0 

    print(f"BẮT ĐẦU CRAWL DỮ LIỆU TỪ {NUM_PAGES} TRANG ĐẦU TIÊN CỦA CHUYÊN MỤC DI SẢN...")
    
    for i in range(1, NUM_PAGES + 1):
        page_url = get_page_url(BASE_URL, i)
        
        # Crawl dữ liệu và cập nhật ID
        articles_on_page, current_id = scrape_page(page_url, current_id)
        
        all_articles.extend(articles_on_page)
        
        # Nghỉ giữa các trang
        time.sleep(3) 

    # Lưu dữ liệu vào CSV
    save_to_csv(all_articles, 'E:/NLP_project/Data/raw.csv')


if __name__ == "__main__":
    main()

BẮT ĐẦU CRAWL DỮ LIỆU TỪ 25 TRANG ĐẦU TIÊN CỦA CHUYÊN MỤC DI SẢN...
--- Đang tải dữ liệu từ trang danh sách: https://vietnamnet.vn/van-hoa-giai-tri/di-san
  > Lấy nội dung bài viết 1/30: Lần đầu tiên Festival Sông Hồng sẽ được tổ chức tạ...
  > Lấy nội dung bài viết 2/30: Những khám phá khảo cổ đặc biệt tại Vườn Chuối...
  > Lấy nội dung bài viết 3/30: Lai lịch xác tàu cổ được cho là bảo vật hiếm ở vùn...
  > Lấy nội dung bài viết 4/30: Tùng Dương và hơn 1.000 diễn viên 'đánh thức' di s...
  > Lấy nội dung bài viết 5/30: Giải mã bí ẩn về đời sống hoàng cung tại Hoàng thà...
  > Lấy nội dung bài viết 6/30: Thành lập Ban Quản lý Di sản thế giới vịnh Hạ Long...
  > Lấy nội dung bài viết 7/30: Lễ hội chùa Cổ Lễ 2025...
  > Lấy nội dung bài viết 8/30: Cảnh tượng lạ tại Lễ hội chùa Keo Hành Thiện ở Nin...
  > Lấy nội dung bài viết 9/30: Tinh hoa di sản Việt hội tụ tại Văn Miếu - Quốc Tử...
  > Lấy nội dung bài viết 10/30: Ngôi cổ tự trăm năm vắng bóng nhà sư ở làng khoa b...
  > Lấy nội dung